## Read and split Docs

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
#load pdf file
loader3 = PyPDFLoader("BodyPartRecognition.pdf") #your pdf file path
papper = loader3.load_and_split()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(papper)

## Generating Embeddings with Ollama

1. Download ollama on your own machine from their official website
2. Then in your terminal use these commands to download a language model and an embedding model:


    ollama run gemma3
    
    ollama pull all-minilm


In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

local_embeddings = OllamaEmbeddings(model="all-minilm")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

## Implementing the Retrieval System

In [ ]:
question = "what is the body part Recognition?"

# take the top 3 most relevant documents according to the question
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

In [ ]:
context = ' '.join([doc.page_content for doc in retrieved_docs])

## Use LLM to create an answer using the selected part of the document and the question.

In [ ]:
from langchain_ollama.llms import OllamaLLM

#llm = OllamaLLM(model="llama3.2:1b")

llm = OllamaLLM(model="gemma3")
response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")

In [ ]:
print(response)